# Qiskit Metal & QDesignOptimizer Tutorial  
## Agenda
1. Quick recap of Qiskit Metal basics
2. QDesignOptimizer architecture overview
3. Explore each project file (`names.py`, `design.py`, …)
4. Run a MiniStudy + optimisation loop
5. Inspect results & update design variables

![alt text](QDesignOpt_HomePage.png)

Code Documentation: https://202q-lab.github.io/QDesignOptimizer/index.html 

Repository link: https://github.com/202Q-lab/QDesignOptimizer

# Installation 
### Installation using pip

`pip install qdesignoptimizer`

`pip install --no-deps qiskit-metal==0.1.5`

### Installation from the github repository 

`git clone https://github.com/202Q-lab/QDesignOptimizer`

`cd QDesignOptimizer`

#### Creating a conda environment

`conda env create -f environment.yml`

`conda activate qdesignenv`

** To run the tutorials, you need to clone the github repository **

# Eigenmode simulation of single qubit with readout resonator

In [1]:
%load_ext autoreload
%autoreload 2

## Rendering the design

In [2]:
from chip_initialization import create_chip_and_gui

design, gui = create_chip_and_gui()

from design import render_qiskit_metal_design

render_qiskit_metal_design(design, gui)

## Creating the study and optimization targets

In [3]:
import names as n
import mini_studies as ms
import optimization_targets as ot

In [4]:
MINI_STUDY_GROUP = n.NBR_1
MINI_STUDY = ms.get_mini_study_qb_res(group=MINI_STUDY_GROUP)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)


opt_targets = ot.get_opt_targets_2qubits_resonator_coupler(
    groups=[MINI_STUDY_GROUP],
    opt_target_qubit_freq=True,
    opt_target_qubit_anharm=True,
    opt_target_resonator_freq=True,
    opt_target_resonator_kappa=True,
    opt_target_resonator_qubit_chi=True,
    use_simple_resonator_qubit_chi_relation=True,
)

## Creating design analysis objects

In [5]:
import time
import parameter_targets as pt
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from design import CoupledLineTee_mesh_names
from qdesignoptimizer.design_analysis_types import MeshingMap

from qdesignoptimizer.utils.utils import get_save_path

# Closing Ansys hfss
from qdesignoptimizer.utils.utils import close_ansys

close_ansys()

In [ ]:
# A qiskit metal object to rebuild layout after variables change
# parameter-target dictionary
# current values of the design-variables
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)

# map for finer meshing
meshing_map = [
    MeshingMap(component_class=CoupledLineTee, mesh_names=CoupledLineTee_mesh_names)
]

# References to the `DesignAnalysisState`, current ministudy and optimization targets
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path=get_save_path("out/", n.CHIP_NAME),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS,
    meshing_map=meshing_map,
)

[INFO|2025-04-24 16:12:18]: self.eig_solver.sim.setup 
{
    "name": "Resonator_setup",
    "reuse_selected_design": true,
    "reuse_setup": true,
    "min_freq_ghz": 1,
    "n_modes": 1,
    "max_delta_f": 0.5,
    "max_passes": 10,
    "min_passes": 1,
    "min_converged": 1,
    "pct_refinement": 30,
    "basis_order": 1,
    "vars": {
        "Lj": "10 nH",
        "Cj": "0 fF"
    }
}
INFO 04:12PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:12PM [load_ansys_project]: 	Opened Ansys App
INFO 04:12PM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 04:12PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/helambe/OneDrive - Chalmers/Documents/Ansoft/
	Project:   Project23
INFO 04:12PM [connect_design]: No active design found (or error getting active design).
INFO 04:12PM [connect]: 	 Connected to project "Project23". No design detected
04:12PM 27s WARNING [activate_ansys_design]: The design_name=get_mini_study_qb_res was not in active proje

## Optimization step

In [7]:
# number of runs of optimization and number of passes for simulation at each run
group_runs = 5
group_passes = 4
delta_f = 0.001
for i in range(group_runs):
    design_analysis.update_nbr_passes(group_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

[INFO|2025-04-24 16:12:29]: Updated_design_vars
{
    "cpw_width": "10 um",
    "cpw_gap": "6 um",
    "design_var_width_qubit_1": "400um",
    "design_var_width_qubit_2": "400um",
    "design_var_cl_pos_x_qubit_1": "-100um",
    "design_var_cl_pos_y_qubit_1": "-300um",
    "design_var_cl_pos_x_qubit_2": "-100um",
    "design_var_cl_pos_y_qubit_2": "300um",
    "design_var_length_resonator_1_capacitance": "20um",
    "design_var_length_resonator_2_capacitance": "20um",
    "design_var_lj_qubit_1": "12.1nH",
    "design_var_lj_qubit_2": "9.1nH",
    "design_var_cj_qubit_1": "0fF",
    "design_var_cj_qubit_2": "0fF",
    "design_var_length_resonator_1": "7500um",
    "design_var_length_resonator_2": "6500um",
    "design_var_length_coupler_1to2": "4000um",
    "design_var_coupl_length_qubit_1_resonator_1": "100um",
    "design_var_coupl_length_qubit_2_resonator_2": "100um",
    "design_var_coupl_length_resonator_1_tee": "400um",
    "design_var_coupl_length_resonator_2_tee": "160um"
}


Initializing new desktop!
PyAEDT INFO: StdOut is enabled
PyAEDT INFO: Log on file is enabled
PyAEDT INFO: Log on Desktop Message Manager is enabled
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT outside AEDT with CPython and PythonNET.
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\AnsysEM21.2\Win64.
PyAEDT INFO: Launching AEDT with COM plugin using PythonNET.
PyAEDT INFO: AEDT 2021.2 Started with process ID 38260.
PyAEDT INFO: pyaedt v0.6.94
PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]
PyAEDT INFO: AEDT 2021.2.0 Build Date 2021-06-02 22:21:55
PyAEDT INFO: No project is defined. Project Project23 exists and has been read.
PyAEDT INFO: Active Design set to get_mini_study_qb_res
PyAEDT INFO: Aedt Objects correctly read


INFO 04:12PM [analyze]: Analyzing setup Setup


Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1

        energy_elec_all       = 1.58690936643332e-23
        energy_elec_substrate = 1.45638314938185e-23
        EPR of substrate = 91.8%

        energy_mag    = 2.21909815161268e-25
        energy_mag % of energy_elec_all  = 1.4%
        

Variation 0  [1/1]

  Mode 0 at 5.92 GHz   [1/2]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               98.6%  7.935e-24  1.11e-25

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 96.76%
	jj_name_qub

WARNING 04:14PM [__init__]: <p>Error: <class 'IndexError'></p>


...

INFO 04:32PM [analyze]: Analyzing setup Setup


Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1

        energy_elec_all       = 5.55196280430578e-24
        energy_elec_substrate = 5.09360545548825e-24
        EPR of substrate = 91.7%

        energy_mag    = 5.53900708661347e-24
        energy_mag % of energy_elec_all  = 99.8%
        

Variation 0  [1/1]

  Mode 0 at 4.19 GHz   [1/2]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               98.7%   2.14e-23 2.693e-25

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 98.12%
	jj_name_qu

WARNING 04:38PM [__init__]: <p>Error: <class 'IndexError'></p>



ANALYSIS DONE. Data saved to:

C:\data-pyEPR\Project23\get_mini_study_qb_res\2025-04-24 16-37-44.npz


	 Differences in variations:



 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Variation 0

Starting the diagonalization
Finished the diagonalization
Pm_norm=
modes
0    1.020719
1    1.014059
dtype: float64

Pm_norm idx =
   jj_name_qubit_1
0             True
1            False
*** P (participation matrix, not normlz.)
   jj_name_qubit_1
0         0.967479
1         0.002360

*** S (sign-bit matrix)
   s_jj_name_qubit_1
0                  1
1                  1
*** P (participation matrix, normalized.)
      0.99
    0.0024

*** Chi matrix O1 PT (MHz)
    Diag is anharmonicity, off diag is full cross-Kerr.
       181     1.24
      1.24  0.00211

*** Chi matrix ND (MHz) 
       200     1.02
      1.02  0.00149

*** Frequencies O1 PT (MHz)
0    4006.710037
1    5987.986207
dtype: float64

*** Frequencies ND (MHz)
0    3997.941709
1    5988.018023
dt

[INFO|2025-04-24 16:38:17]: freq_ND_results
{
    "0": {
        "0": 3997.9417093998104,
        "1": 5988.018022535135
    }
}
04:38PM 17s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  CRITICAL: QEventDispatcherWin32::wakeUp: Failed to post a message (Not enough quota is available to process this command.)



## Results

### Eigenmodes

In [8]:
design_analysis.get_eigenmode_results()

Freq. (GHz)  Quality Factor  Kappas (kHz)    Freq. (Hz)  \
variation mode                                                            
0         0        4.188451    1.888898e+06      2.217404  4.188451e+09   
          1        5.988607    8.895133e+03    673.245382  5.988607e+09   

                  Kappas (Hz)  
variation mode                 
0         0       2217.404071  
          1     673245.382078

### Cross Kerr

In [9]:
design_analysis.get_cross_kerr_matrix(iteration=-1)

0         1
variation                        
0         0  199.877522  1.023817
          1    1.023817  0.001486

## Update parameters

In [10]:
design_analysis.overwrite_parameters()

[INFO|2025-04-24 16:38:18]: Overwritten parameters
{
    "design_var_lj_qubit_1": "13.844405883699757 nH",
    "design_var_width_qubit_1": "833.1561025250245 um",
    "design_var_length_resonator_1": "8202.24862885452 um",
    "design_var_coupl_length_resonator_1_tee": "999.99999999 um",
    "design_var_coupl_length_qubit_1_resonator_1": "413.69915175576773 um"
}


## Close

In [11]:
# close_ansys()